In [1]:
from itertools import product
import pandas as pd
import math

def truth_table(vector):
    n = int(math.log2(len(vector)))
    inputs = list(product([0, 1], repeat=n))
    columns = [f'x{i+1}' for i in range(n)] + ['F']
    rows = [list(inp) + [vector[i]] for i, inp in enumerate(inputs)]
    return pd.DataFrame(rows, columns=columns)

def zhegalkin_polynomial(vector):
    n = int(math.log2(len(vector)))
    coefficients = list(map(int, vector))  # Копируем значения вектора
    terms = []
    inputs = list(product([0, 1], repeat=n))

    for level in range(len(vector)):
        if coefficients[0]:  # Если старший коэффициент != 0
            term = []
            for i in range(n):
                if inputs[level][i]:
                    term.append(f'x{i+1}')  # Преобразуем индексы в x1, x2, ...
            terms.append("".join(term) if term else "1")
        # Обновляем коэффициенты разностей по модулю 2
        for i in range(len(coefficients) - 1):
            coefficients[i] = (coefficients[i] + coefficients[i + 1]) % 2
        coefficients.pop()

    polynomial = " ⊕ ".join(terms)  # Используем символ ⊕ для XOR (⊕)
    is_linear = all(len(term.split('x')) <= 2 for term in terms)  # Линейна, если все члены имеют <=1 переменную
    return polynomial, is_linear

def preserves_zero(vector):
    return vector[0] == '0'

def preserves_one(vector):
    return vector[-1] == '1'

def is_self_dual(vector):
    n = len(vector)
    return all(vector[i] != vector[n - 1 - i] for i in range(n))

def is_monotonic(vector):
    n = int(math.log2(len(vector)))
    inputs = list(product([0, 1], repeat=n))
    for i, x in enumerate(inputs):
        for j, y in enumerate(inputs):
            if all(x[k] <= y[k] for k in range(n)) and vector[i] > vector[j]:
                return False
    return True

def is_system_complete(vectors):
    properties = {
        "сохраняет ноль": False,
        "сохраняет единицу": False,
        "самодвойственная": False,
        "монотонная": False,
        "линейная": False,
    }

    for vector in vectors:
        _, is_lin = zhegalkin_polynomial(vector)
        if not preserves_zero(vector):
            properties["сохраняет ноль"] = True
        if not preserves_one(vector):
            properties["сохраняет единицу"] = True
        if not is_self_dual(vector):
            properties["самодвойственная"] = True
        if not is_monotonic(vector):
            properties["монотонная"] = True
        if not is_lin:
            properties["линейная"] = True

    return all(properties.values())

def main():
    with open("file.txt", "r") as file:
        vectors = [line.strip() for line in file]

    results = []

    for idx, vector in enumerate(vectors):
        print(f"\nФункция {idx + 1}:")
        print(f"Вектор истинности: {vector}")

        table = truth_table(vector)
        print("Таблица истинности:")
        print(table.to_string(index=False))

        polynomial, is_lin = zhegalkin_polynomial(vector)
        print(f"Полином Жегалкина: {polynomial}")
        print("Таблица классов:")
        print(f"Самодвойственная: {'+' if is_self_dual(vector) else '-'}")
        print(f"Монотонная: {'+' if is_monotonic(vector) else '-'}")
        print(f"Линейная: {'+' if is_lin else '-'}")
        print(f"Сохраняет ноль: {'+' if preserves_zero(vector) else '-'}")
        print(f"Сохраняет единицу: {'+' if preserves_one(vector) else '-'}")

        results.append({
            "Вектор": vector,
            "Самодвойственная": '+' if is_self_dual(vector) else '-',
            "Монотонная": '+' if is_monotonic(vector) else '-',
            "Линейная": '+' if is_lin else '-',
            "Сохраняет ноль": '+' if preserves_zero(vector) else '-',
            "Сохраняет единицу": '+' if preserves_one(vector) else '-',
            "Полином Жегалкина": polynomial
        })

    results_df = pd.DataFrame(results)
    print("\nИтоговая таблица:")
    print(results_df.to_string(index=False))

    if is_system_complete(vectors):
        print("\nСистема функций полна.")
    else:
        print("\nСистема функций не полна.")

if __name__ == "__main__":
    main()



Функция 1:
Вектор истинности: 1100
Таблица истинности:
 x1  x2 F
  0   0 1
  0   1 1
  1   0 0
  1   1 0
Полином Жегалкина: 1 ⊕ x1
Таблица классов:
Самодвойственная: +
Монотонная: -
Линейная: +
Сохраняет ноль: -
Сохраняет единицу: -

Функция 2:
Вектор истинности: 0111
Таблица истинности:
 x1  x2 F
  0   0 0
  0   1 1
  1   0 1
  1   1 1
Полином Жегалкина: x2 ⊕ x1 ⊕ x1x2
Таблица классов:
Самодвойственная: -
Монотонная: +
Линейная: -
Сохраняет ноль: +
Сохраняет единицу: +

Функция 3:
Вектор истинности: 1110
Таблица истинности:
 x1  x2 F
  0   0 1
  0   1 1
  1   0 1
  1   1 0
Полином Жегалкина: 1 ⊕ x1x2
Таблица классов:
Самодвойственная: -
Монотонная: -
Линейная: -
Сохраняет ноль: -
Сохраняет единицу: -

Функция 4:
Вектор истинности: 10101010
Таблица истинности:
 x1  x2  x3 F
  0   0   0 1
  0   0   1 0
  0   1   0 1
  0   1   1 0
  1   0   0 1
  1   0   1 0
  1   1   0 1
  1   1   1 0
Полином Жегалкина: 1 ⊕ x3
Таблица классов:
Самодвойственная: +
Монотонная: -
Линейная: +
Сохраняет нол